# 1. Preparation

In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
OUTPUT_DIR = '../output_data'

## 1.A. Common Functions

In [3]:
def merge_id(prev_table, keys_table, name):
    merged_df = prev_table.merge(keys_table, left_on=name, right_on=name, how='left')
    merged_df = merged_df.rename(columns={'id': f'{name}_id'})
    merged_df = merged_df.drop(columns=[name])
    merged_df[f'{name}_id'] = merged_df[f'{name}_id'].where(merged_df[f'{name}_id'].notna(), 0).astype(int)
    return merged_df

In [4]:
def cleanup_semicolon(source):
    with open(source, 'r') as file:
        content = file.read()
    updated_content = content.replace(';', '')
    with open(source, 'w') as file:
        file.write(updated_content)

## 1.B. Key Table Generator

In [5]:
def create_table_key(dataframe, column):
    file_path = f'{OUTPUT_DIR}/key_{column}.csv'
    new_table = pd.DataFrame(
        dataframe[column].unique(),
        columns=[column]
    ).dropna().sort_values(column).reset_index(drop=True)
    
    # If the file already exists, load it
    if os.path.exists(file_path):
        existing_table = pd.read_csv(file_path)
        # Find the new values that are not in the existing table
        new_values = new_table[~new_table[column].isin(existing_table[column])]
        if not new_values.empty:
            # Assign IDs to the new values, starting after the max existing ID
            max_id = existing_table['id'].max()
            new_values['id'] = range(max_id + 1, max_id + 1 + len(new_values))
            # Append the new values to the existing table
            updated_table = pd.concat([existing_table, new_values], ignore_index=True)
        else:
            updated_table = existing_table  # No new values to add, keep existing table as is
    else:
        # If the file doesn't exist, create new IDs starting from 1
        new_table['id'] = range(1, len(new_table) + 1)
        updated_table = new_table
    updated_table[['id', column]].to_csv(file_path, index=False)
    return updated_table

# 2. JMP Dataset

In [6]:
JMP_INPUT_FILE = '../input_data/JMP/jmp.csv'
JMP_OUTPUT_FILE = f'{OUTPUT_DIR}/table_jmp.csv'

In [7]:
data = pd.read_csv(JMP_INPUT_FILE, encoding='latin-1')
data.head()

,"COUNTRY, AREA OR TERRITORY",Year,Type,TOTAL - At least basic,TOTAL - Annual rate of change in \nat least basic,TOTAL - Safely managed,TOTAL - Annual rate of change in safely managed,"TOTAL - Annual rate of change SM, manual calculation","TOTAL - Annual rate of change ALB, manual calculation"
0,Afghanistan,2000,Water,27.4,2.5,11.1,0.9,-99.0,-99.0
1,Afghanistan,2001,Water,27.5,2.5,11.1,0.9,0.0,0.0
2,Afghanistan,2002,Water,29.7,2.5,12.0,0.9,0.9,2.2
3,Afghanistan,2003,Water,31.9,2.5,12.9,0.9,0.9,2.2
4,Afghanistan,2004,Water,34.1,2.5,13.8,0.9,0.9,2.2


## 2.A. JMP Data Processing

### 2.A.1 Rename the columns

In [8]:
data.columns = [
    'country',
    'year',
    'jmp_name',
    'total_ALB',
    'annual_rate_change_ALB',
    'total_SM',
    'annual_rate_change_SM',
    'manual_rate_change_SM',
    'manual_rate_change_ALB'
]
data.head()

,country,year,jmp_name,total_ALB,annual_rate_change_ALB,total_SM,annual_rate_change_SM,manual_rate_change_SM,manual_rate_change_ALB
0,Afghanistan,2000,Water,27.4,2.5,11.1,0.9,-99.0,-99.0
1,Afghanistan,2001,Water,27.5,2.5,11.1,0.9,0.0,0.0
2,Afghanistan,2002,Water,29.7,2.5,12.0,0.9,0.9,2.2
3,Afghanistan,2003,Water,31.9,2.5,12.9,0.9,0.9,2.2
4,Afghanistan,2004,Water,34.1,2.5,13.8,0.9,0.9,2.2


### 2.A.2. Categorize the Values

In [9]:
data_melted = pd.melt(
    data, 
    id_vars=['country', 'year', 'jmp_name'],  # columns to keep
    var_name='variable',  # melted
    value_name='value' # values
)
data_melted['value_type'] = data_melted['variable'].apply(lambda x: 'total' if 'total' in x else 'annual_rate_change')
data_melted['jmp_category'] = data_melted['variable'].apply(lambda x: 'ALB' if 'ALB' in x else 'SM')
data_melted = data_melted.drop(columns=['variable'])
data_melted.head()

,country,year,jmp_name,value,value_type,jmp_category
0,Afghanistan,2000,Water,27.4,total,ALB
1,Afghanistan,2001,Water,27.5,total,ALB
2,Afghanistan,2002,Water,29.7,total,ALB
3,Afghanistan,2003,Water,31.9,total,ALB
4,Afghanistan,2004,Water,34.1,total,ALB


## 2.B. JMP Table Keys

### 2.B.1. Countries

In [10]:
countries_table = create_table_key(data_melted, 'country')
countries_table

,country,id
0,Afghanistan,1
1,Albania,2
2,Algeria,3
3,American Samoa,4
4,Andorra,5
...,...,...
228,Wallis and Futuna Islands,229
229,Western Sahara,230
230,Yemen,231
231,Zambia,232


### 2.B.1. JMP Names

In [11]:
jmp_names_table = create_table_key(data_melted, 'jmp_name')
jmp_names_table

,jmp_name,id
0,Sanitation,1
1,Water,2


### 2.B.2. JMP Categories

In [12]:
jmp_categories_table = create_table_key(data_melted, 'jmp_category')
jmp_categories_table

,jmp_category,id
0,ALB,1
1,SM,2


### 2.B.3. JMP Value Types

In [13]:
value_types_table = create_table_key(data_melted, 'value_type')
value_types_table

,value_type,id
0,annual_rate_change,1
1,total,2


## 2.C. JMP Table Results

### 2.C.1. JMP Key Table Mapping

In [14]:
table_with_id = merge_id(data_melted, value_types_table, 'value_type')
table_with_id = merge_id(table_with_id, countries_table, 'country')
table_with_id = merge_id(table_with_id, jmp_names_table, 'jmp_name')
table_with_id = merge_id(table_with_id, jmp_categories_table, 'jmp_category')

### 2.C.2. JMP Final Result

In [15]:
table_with_id.head()

,year,value,value_type_id,country_id,jmp_name_id,jmp_category_id
0,2000,27.4,2,1,2,1
1,2001,27.5,2,1,2,1
2,2002,29.7,2,1,2,1
3,2003,31.9,2,1,2,1
4,2004,34.1,2,1,2,1


### 2.C.3. Save JMP Table

In [16]:
table_with_id.to_csv(JMP_OUTPUT_FILE, index=False)

# 3. IFS Dataset

In [17]:
IFS_INPUT_DIR = '../input_data/IFs'
IFS_OUTPUT_FILE = f'{OUTPUT_DIR}/table_ifs.csv'
final_columns = ['indicator','year','country','unit','value_name','jmp_category','commitment','value']

In [18]:
files_to_keep = [
    "01. Deaths by Category of Cause - Millions (2nd Dimensions = Diarrhea).csv",
    "11. Governance Effectiveness - WB index.csv",
    "12. Value Added by Sector, Currency - Billion dollars.csv",
    "13. Sanitation Services, Access, percent of population (2nd Dimensions = Basic + Safely Managed).csv",
    "14. Sanitation Services, Access, Number of people, million (2nd Dimensions = Basic + Safely Managed).csv",
    "15. Sanitation Services, Expenditure, Capital, Billion $ (2nd Dimensions = Basic + Safely Managed).csv",
    "16. Sanitation Services, Expenditure, Maintenance, Billion $ (2nd Dimensions = Basic + Safely Managed).csv",
    "17. Water Services, Access, percent of population (2nd Dimension = Basic + Safely Managed).csv",
    "18. Water Services, Access, Number of people, million (2nd Dimensions = Basic + Safely Managed).csv",
    "19. Water Services, Expenditure, Capital, Billion $ (2nd Dimensions = Basic + Safely Managed).csv",
    "20. Water Services, Expenditure, Maintenance, Billion $ (2nd Dimensions = Basic + Safely Managed).csv",
    "21. Population - Millions.csv",
    "23. GDP (PPP) - Billion dollars.csv",
    "24. Stunted children, History and Forecast - Million.csv",
    "25. Population under 5 Years, Headcount - Millions.csv",
    "26. Malnourished Children, Headcount - Millions.csv"
]
year_filter_config = {
    "year_range": {
        "years": list(range(2019, 2050)),
        "files": [
            "13. Sanitation Services, Access, percent of population (2nd Dimensions = Basic + Safely Managed).csv",
            "17. Water Services, Access, percent of population (2nd Dimension = Basic + Safely Managed).csv"
        ]
    },
    "milestone_years": [2030, 2050]
}

In [19]:
files = [
    f"{IFS_INPUT_DIR}/{f}" for f in os.listdir(IFS_INPUT_DIR)
    if os.path.isfile(os.path.join(IFS_INPUT_DIR, f))
]
files = [f"{IFS_INPUT_DIR}/{file}" for file in files_to_keep]

## 3.A. IFS Functions

In [20]:
def get_ifs_name(source):
    source = re.sub(r"\s*\(2nd Dimension.*?\)", "", source)
    return re.sub(r'^\d+\. ', '', source.replace(f"{IFS_INPUT_DIR}/", "")).replace(".csv", "")

In [21]:
def get_value_types(lst):
    lst = lst.split('.')[0]
    lst = lst.replace('_0_','_0.').split("_")
    return lst

In [22]:
def cleanup_data(dataframe):
    dataframe['unit'] = dataframe['unit'].apply(lambda x: x.replace("2017","") if x else None)
    dataframe['value'] = dataframe['value'].apply(lambda x: x.replace(' ','') if ' ' in str(x) else x)
    dataframe['value'] = dataframe['value'].apply(lambda x: x if len(str(x)) > 0 else np.nan)

## 3.B. IFS Data Processing

In [23]:
combined_df = pd.DataFrame(columns=final_columns)
for file in files:
    print(f"Processing {file}")
    cleanup_semicolon(file)
    data = pd.read_csv(file, header=[1,4,5], sep=',')
    new_columns = list(data.columns)
    for i, col in enumerate(new_columns):
        if col == ('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2'):
            new_columns[i] = 'Year'
    data.columns = new_columns
    df = pd.DataFrame(data.to_dict('records'))
    df_melted = df.melt(id_vars=['Year'], var_name='variable', value_name='value')
    new_data = []
    for value_list in df_melted.to_dict('records'):
        value_type = get_value_types(value_list["variable"][2])
        new_data.append({
            "year": value_list["Year"],
            "country": value_list["variable"][0],
            "unit": value_list["variable"][1],
            "value_type": list(filter(lambda v:v,value_type)),
            "value": value_list["value"]
        })
    df = pd.DataFrame(new_data)
    df_split = pd.DataFrame(df['value_type'].tolist(), index=df.index)
    df_split.columns = ['value_name', 'jmp_category', 'commitment']
    df_final = pd.concat([df, df_split], axis=1)
    df_final['indicator'] = get_ifs_name(file)
    df_final = df_final[final_columns]
    combined_df = pd.concat([combined_df.dropna(axis=1, how='all'), df_final], ignore_index=True)

Processing ../input_data/IFs/01. Deaths by Category of Cause - Millions (2nd Dimensions = Diarrhea).csv
Processing ../input_data/IFs/11. Governance Effectiveness - WB index.csv
Processing ../input_data/IFs/12. Value Added by Sector, Currency - Billion dollars.csv
Processing ../input_data/IFs/13. Sanitation Services, Access, percent of population (2nd Dimensions = Basic + Safely Managed).csv
Processing ../input_data/IFs/14. Sanitation Services, Access, Number of people, million (2nd Dimensions = Basic + Safely Managed).csv
Processing ../input_data/IFs/15. Sanitation Services, Expenditure, Capital, Billion $ (2nd Dimensions = Basic + Safely Managed).csv
Processing ../input_data/IFs/16. Sanitation Services, Expenditure, Maintenance, Billion $ (2nd Dimensions = Basic + Safely Managed).csv
Processing ../input_data/IFs/17. Water Services, Access, percent of population (2nd Dimension = Basic + Safely Managed).csv
Processing ../input_data/IFs/18. Water Services, Access, Number of people, milli

In [24]:
cleanup_data(combined_df)
combined_df.head()

,indicator,year,country,unit,value_name,jmp_category,commitment,value
0,Deaths by Category of Cause - Millions,2019,All countries WHHS Tool1,Mil People,Base,None,None,1.33
1,Deaths by Category of Cause - Millions,2020,All countries WHHS Tool1,Mil People,Base,None,None,1.358
2,Deaths by Category of Cause - Millions,2021,All countries WHHS Tool1,Mil People,Base,None,None,1.332
3,Deaths by Category of Cause - Millions,2022,All countries WHHS Tool1,Mil People,Base,None,None,1.307
4,Deaths by Category of Cause - Millions,2023,All countries WHHS Tool1,Mil People,Base,None,None,1.295


**To check the results before merging with the ID, please run the following command:**

In [25]:
# combined_df.to_csv("./testing.csv",index=False)

## 3.C. IFS Table of Keys

### 3.C.1. Indicators

In [26]:
indicator_table = create_table_key(combined_df, 'indicator')
indicator_table

,indicator,id
0,Deaths by Category of Cause - Millions,1
1,GDP (PPP) - Billion dollars,2
2,Governance Effectiveness - WB index,3
3,"Malnourished Children, Headcount - Millions",4
4,Population - Millions,5
5,"Population under 5 Years, Headcount - Millions",6
6,"Sanitation Services, Access, Number of people,...",7
7,"Sanitation Services, Access, percent of popula...",8
8,"Sanitation Services, Expenditure, Capital, Bil...",9
9,"Sanitation Services, Expenditure, Maintenance,...",10


### 3.C.2. Units

In [27]:
units_table = create_table_key(combined_df, 'unit')
units_table

,unit,id
0,Billion $,1
1,Index 0-5,2
2,Mil People,3
3,Million,4
4,Percent,5
5,Trillion $,6


### 3.C.3. Value Names

In [28]:
value_names_table = create_table_key(combined_df, 'value_name')
value_names_table

,value_name,id
0,Base,1
1,FS,2
2,FW,3
3,FWS,4
4,SI,5
5,WI,6
6,WSI,7


### 3.C.4. JMP Categories (Retry)

In [29]:
jmp_categories_table = create_table_key(combined_df, 'jmp_category')
jmp_categories_table

/tmp/ipykernel_898281/940121742.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_values['id'] = range(max_id + 1, max_id + 1 + len(new_values))


,id,jmp_category
0,1,ALB
1,2,SM
2,3,BS


### 3.C.5. Commitments

In [30]:
commitments_table = create_table_key(combined_df, 'commitment')
commitments_table

,commitment,id
0,0.5x,1
1,2030,2
2,2050,3
3,2x,4
4,4x,5
5,6x,6


### 3.C.6. Country

In [31]:
countries_table = create_table_key(combined_df, 'country')
countries_table

/tmp/ipykernel_898281/940121742.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_values['id'] = range(max_id + 1, max_id + 1 + len(new_values))


,id,country
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,American Samoa
4,5,Andorra
...,...,...
232,233,Zimbabwe
233,234,All countries WHHS Tool1
234,235,Congo Dem. Republic of the
235,236,Sudan South


## 3.D. IFS Table Results

### 3.D.1. Custom Table Mapping (JMP)

- FS = Full Sanitation Access
- FW = Full Water Access
- FWS = Full Water and Sanitation Access
- SI = Sanitation Increased
- WI = Water Increased
- WSI = Water and Sanitation Increased

In [32]:
if jmp_names_table['id'].shape[0] == 2:
    new_jmp_id = jmp_names_table['id'].max() + 1
    new_row = pd.DataFrame([{"id":new_jmp_id, "jmp_name": "Water and Sanitation"}])
    jmp_names_table = pd.concat([jmp_names_table, new_row], ignore_index=True)
    jmp_names_table.to_csv(f"{OUTPUT_DIR}/key_jmp_name.csv", index=False)

jmp_dict = dict(zip(jmp_names_table['jmp_name'], jmp_names_table['id']))

In [33]:
def map_jmp_id(jmp_type):
    id = 0
    if 'W' in jmp_type:  # Water is indicated by 'W' in the type
        id = jmp_dict['Water']
    if 'S' in jmp_type:  # Sanitation is indicated by 'S' in the type
        id = jmp_dict['Sanitation']
    # if W and S is available then create new category "Water and Sanitation"
    if 'W' in jmp_type and 'S' in jmp_type:
        id = jmp_dict['Water and Sanitation']
    return id

In [34]:
combined_df['jmp_name_id'] = combined_df['value_name'].apply(map_jmp_id)
combined_df.tail(2)

,indicator,year,country,unit,value_name,jmp_category,commitment,value,jmp_name_id
1724975,"Malnourished Children, Headcount - Millions",2099,Zambia,Mil People,WSI,SM,6x,0.051,3
1724976,"Malnourished Children, Headcount - Millions",2100,Zambia,Mil People,WSI,SM,6x,0.05,3


### 3.D.2. IFS Key Table Mapping

In [35]:
table_with_id = merge_id(combined_df, indicator_table, 'indicator')
table_with_id = merge_id(table_with_id, units_table, 'unit')
table_with_id = merge_id(table_with_id, value_names_table, 'value_name')
table_with_id = merge_id(table_with_id, jmp_categories_table, 'jmp_category')
table_with_id = merge_id(table_with_id, commitments_table, 'commitment')
table_with_id = merge_id(table_with_id, countries_table, 'country')

### 3.D.3. IFS Final Result

In [36]:
table_with_id = table_with_id[table_with_id['value'].notna()]
table_with_id = table_with_id.sort_values('year')
table_with_id.reset_index(drop=True).tail()

,year,value,jmp_name_id,indicator_id,unit_id,value_name_id,jmp_category_id,commitment_id,country_id
1682977,2100,0.413,1,9,1,2,2,2,185
1682978,2100,0.0,1,9,1,2,1,3,185
1682979,2100,0.0,1,9,1,2,1,2,185
1682980,2100,60.44,1,6,3,5,3,1,98
1682981,2100,0.05,3,4,3,7,2,6,232


### 3.D.2. Save IFS Table

In [37]:
table_with_id.to_csv(IFS_OUTPUT_FILE, index=False)